# Herança e Composição

Herança e composição são dois conceitos fundamentais na orientação a objetos que modelam **relacionamentos** entre duas classes.

## Herança

A herança é um conceito modela uma relação do tipo **"isto é"** (is a). Logo, _classes derivadas_ (também chamadas de subclasses) de uma _classe base_ são especializações desta classe base (também chamada de super classe).

A Herança é usada principalmente para **incrementar** ou **especializar** funcionalidades de uma classe base, enquanto as demais funcionalidades permanecem iguais. Isso significa que podemos:

- Adicionar comportamentos
- Modificar comportamentos

Em Python, a sintaxe de uma herança é da forma `class <SubClassName>(<SuperClassName>):`

In [14]:
class Animal:
    """Classe base: Animal"""
    def walk(self):
        print("I'm an animal and I walk")

class Cachorro(Animal):
    """Classe derivada: Cachorro"""
    def play(self):
        print("I'm an dog and I like to chase balls")

In [10]:
c = Cachorro()
c.walk()

I'm an animal and I walk


In [12]:
# Diferente de instâncias da classe Animal, os da classe cachoro possuem um comportamento adicional "play" (brincar)
#
c.play()

I'm an dog and I like to chase balls


### `super()`

Há diversas situações onde, a fim de evitar duplicação de código, queremos aproveitar de funcionalidades já implementadas na classe base. Por exemplo, considere a seguinte situação:

In [8]:
class Animal:

    def __init__(self, weight):
        self.weight = weight


class Cachorro(Animal):

    def __init__(self, weight, sound):
        self.weight = weight
        self.sound = sound

Note que na classe `Cachorro`, não é interessante replicarmos a lógica de atribução do dado `self.weight`. A melhor abordagem seria aproveitar da lógica já existente na classe base.

Podemos fazer isso usando a função `super()`. A `super()` é uma função que retorna uma instância da classe base, permitindo-nos chamar diretamente seus métodos.

O uso mais comum de `super()` é para inicializar atributos definidos na classe base.

In [15]:
class Animal:
    
    def __init__(self, weight):
        self.weight = weight


class Cachorro(Animal):
    
    def __init__(self, weight, sound):
        super().__init__(weight)
        self.sound = sound

In [16]:
animal = Animal(10)
animal.weight

10

In [17]:
cachorro = Cachorro(10, "Au Au")
cachorro.weight, cachorro.sound

(10, 'Au Au')

### Sobrecarga de Métodos

Além de adicionar novos comportamentos à classes, podemos querer modificar comportamentos já existentes.

Por exemplo, podemos considerar que todo animal emite um som. Porém, o som de um gato é diferente de um cachorro. Logo, ambas entidades implementam um método `sound()`, mas cada método executa uma lógica diferente.

In [64]:
class Animal:
    def sound(self):
        print("I'm an animal!")

class Cachorro(Animal):
    def sound(self):
        print("Au au")

In [65]:
animal = Animal()
animal.sound()

I'm an animal!


In [66]:
cachorro = Cachorro()
cachorro.sound()

Au au


Tal estratégia de especializar comportamentos de acordo com cada subclasse é denominada polimorfismo (ad-hoc, uma vez que existem outros tipos de polimorfismo) e é utilizada com a ideia de fornecer uma interface (no caso, a assinatura do método) única com o qual diferentes tipos podem interagir.

Contudo, dado que Python é uma linguagem de tipagem dinâmica, pouco importa o tipo do objeto passado como argumento. Seja `Animal` ou não, contanto que implemente o método desejado, podemos invocá-lo.

Isso significa que o uso de herança continua sendo interessante para o compartilhamento de um código único entre diversas subclasses. Contudo, caso o código compartilhado seja apenas uma interface pública, podemos reconsiderar a implementação aproveitando do *duck typing*.

Por outro lado, é fato que mesmo que um objeto satisfaça uma interface em particular, isso não significa que ele irá funcionar em todas as situações. Portanto, a escolha entre herança, polimorfismo e *duck typing* precisa ser bem pensada.

In [29]:
class Pato:
    def sound(self):
        print("Quack quack")

In [30]:
def make_sound(animal):
    animal.sound()

In [31]:
make_sound(cachorro)

Au au


In [32]:
pato = Pato()
make_sound(pato)

Quack quack


### Classes Abstratas

Embora o *duck typing* seja útil, nem sempre é fácil ou manutenível criar classes que contém todos os comportamentos necessários para o funcionameneto do código. Nesse contexto, podemos definir interfaces de forma mais rígida, que é o caso das classes base abstratas.

Classes base abstratas (ou ABCs, do inglês Abstract Base Classes) são classes que definem um conjunto mínimo (e fundamental) de métodos e atributos que uma classe deve implementar. Com isso, podemos criar classes derivadas que possuem sua própria implementação de cada funcionalidade da classe abstrata base.

> Note que a classe base não pode ser instanciada, uma vez que deve servir apenas como um modelo (template)
>
> Além disso, as classes derivadas **precisam** implementar todas as funcionalidades definidas na classe abstrata base. 

In [40]:
from abc import ABC, abstractmethod

class Animal(ABC):
    
    def __init__(self, specie):
        self.specie = specie
    
    @abstractmethod
    def sound(self):
        pass

    
class Cachorro(Animal):
    
    def __init__(self):
        super().__init__('cachorro')

    def sound(self):
        print("Au au")

In [41]:
cachorro = Cachorro()
cachorro.sound()

Au au


In [42]:
# Note que a classe abstrata não pode ser instanciada
#
animal = Animal('animal')

TypeError: Can't instantiate abstract class Animal with abstract method sound

In [45]:
class Gato(Animal):

    def __init__(self):
        super().__init__('gato')

In [47]:
# Assim como não podemos instanciar uma classe
# que não possui todos os métodos abstratos da classe base redefinidos
#
gato = Gato()

TypeError: Can't instantiate abstract class Gato with abstract method sound

In [48]:
class Gato(Animal):

    def __init__(self):
        super().__init__('gato')
    
    def sound(self):
        print("Miau miau")

In [50]:
gato = Gato()
gato.sound()

Miau miau


Note ainda que, do ponto de vista sintático, não precisamos definir uma classe como sendo abstrata para definir métodos que são abstratos. Contudo, isso abre a possibilidade de subclasses não precisarem redefinir os métodos abstratos que, em teoria, é errado.

In [58]:
from abc import ABC, abstractmethod

class Animal:
    
    def __init__(self, specie):
        self.specie = specie
    
    @abstractmethod
    def sound(self):
        pass

    
class Cachorro(Animal):
    
    def __init__(self):
        super().__init__('cachorro')

    def sound(self):
        print("Au au")


class Gato(Animal):

    def __init__(self):
        super().__init__('gato')

In [61]:
cachorro = Cachorro()
gato = Gato()

gato.sound()

## Composição

A composição modela uma relação do tipo **"composto de"** (has a). Isto significa que a classe em questão é composta (ou seja, contém dados/atributos) pela combinação de diferentes objetos. No geral, chamamos essa instância que compõe a classe de _componente_.

In [1]:
class Motor:
    pass


class Carro:
    
    def __init__(self, cor, modelo, ano):
        self.__cor = cor
        self.__modelo = modelo
        self.__ano = ano
        self.__motor = Motor()

## Exemplos

## Conclusão

### Herança
Herança é um mecanismo que utilizamos para modelarmos situações de **"isto é"** (por exemplo, um cachorro é um Animal, um gato é um Animal) de forma que comportamentos padrões entre todas as subclasses de uma superclasse são herdados sem necessidade de reescrita de código (evitando assim duplicação). Ainda, também podemos tornar especícifico certos comportamentos para cada subclasse.

> Logo, note que o foco do uso de herança está em: organização lógica das entidades e minimização de duplicidade de código.

Além disso, a herança tem dois grandes problemas principais:

#### Interface vs Implementanção

Quando uma subclasse herda da superclasse, são herdadas tanto a interface (assinatura dos métodos, escrita e acesso a atributos) quanto a implementação (código contido nos métodos e que manipulam os atributos de fato). Porém, ao mesmo tempo que queremos reaproveitar o código herdado, também queremos definir múltiplas interfaces para lidar com diferentes situações.

Na maioria das linguagens de programação, como Java, interfaces precisam ser declaradas explícitamente. Já em Python, devido a sua a natureza de tipagem dinâmica, o tipo do objeto pouco importa para a invocação de certos comportamentos. 

Tal conceito é conhecido como duck typing e nada mais é que um estilo de programação (possível em linguagens dinamicamente tipadas) onde: contanto que o objeto tenha o método a ser chamado, então podemos chamá-lo.

> O termo *"Duck typing"* vem da expressão "Se parece um pato e fala como um pato, então provavelmente é um pato"

```python
class Animal:
    def sound(self):
        print("I'm an animal")

class Dog(Animal):
    def sound(self):
        print("I'm a dog")

class Duck:
    def sound(self):
        print("I'm a duck")
```

Contudo, ainda assim podemos querer definir interfaces e, nesta situação, devemos utilizar classes base abstratas que não podem ser instanciadas e obrigam todos os métodos abstratos serem implementados nas subclasses.

#### Explosão de Classes

Um grande problema do uso de herança está na complexa hierarquia de classes que pode-se chegar enquanto o código é escrito e/ou pensado. Afinal, podemos nos encontrar em situação onde queremos criar uma nova (sub)classe pra cada variação existente. Esse é um "design anti-pattern" (algo como Anti Padrão de Projeto) que deve ser evitado a todo custo!

Existem algumas estratégias que podem ser utilizadas para evitar, como é o caso de alguns design patterns ou o uso de composição

### Composição

Já a composição [...]


## Apêndice

Existe um "princípio de programação" (ou design patterns): "Composição prevalece sobre Herança"